# Homework 2
## Social Media Analytics

Clarissa Franklin, Kyle Katzen, Paige McKenzie, Meyappan Subbaiah

In [1]:
import pandas as pd
import os

In [23]:
df = pd.DataFrame(columns=['year', 'month', 'day', 'speaker', 'text'])
for filename in os.listdir("data"):
    with open(os.path.join("data", filename), 'r') as book:
        s = pd.Series(filename.split(' ',1)[0].split('-')+[filename.split(' ',1)[1][:-4], book.read()], 
                      index=['year', 'month', 'day', 'speaker', 'text'])
        df=df.append(s, ignore_index=True)
df.head()

,year,month,day,speaker,text
0,1789,04,30,George Washington,Fellow Citizens of the Senate and the House of...
1,1789,10,03,George Washington,Whereas it is the duty of all Nations to ackno...
2,1790,01,08,George Washington,Fellow Citizens of the Senate and House of Rep...
3,1790,12,08,George Washington,Fellow citizens of the Senate and House of Rep...
4,1790,12,29,George Washington,I the President of the United States by my own...
